In [1]:
from enum import Enum
import random
import skills

In [97]:
class Pool:
    def __init__(self, maximum=0, edge=0):
        self.max = maximum
        self.current = maximum
        self.edge = edge
    def __repr__(self):
        return "{}/{}".format(self.current, self.max)
    def reduce(self, amount):
        if(self.current > 0):
            difference = (self.current - amount)
            if(difference >= 0):
                self.current -= amount
                return 0
            else:
                self.current = 0
                return difference
            #self.current -= amount
            #self.current = max(self.current, 0)
            #return self.current
        else:
            return amount
    def increase(self, amount):
        if(self.current < self.max):
            self.current += amount
            self.current = min(self.current, self.max)
    def effort(self, num):
        spent = (num*3) - self.edge
        if(spent <= self.current):
            self.reduce(spent)
            return True
        else:
            print("Not enough to spend {} effort.\nCurrent: {}\nRequired: {}".format(num, self.current, spent))
            return False

In [2]:
class Stat(Enum):
    Might = 0
    Speed = 1
    Intellect = 2

# Skills List

* Performance
* Bluff
* Insight
* Diplomacy
* Athletics: Might
* Athletics: Speed
* Perception
* Healing
* Survival
* Stealth
* Investigation



* Might Attack
* Speed Attack (Ranged)
* Speed Attack (Melee)
* Intellect Attack
* Might Defense
* Speed Defense
* Intellect Defense
* Initiative


* Numenera/Magic/Occult
* Craft: _

**Social**

* Perform: Stat.Intellect

* Bluff: Stat.Intellect

* Insight: Stat.Intellect

* Diplomacy: Stat.Intellect



**Adventuring**

* Athletics: Stat.Might

* Acrobatics: Stat.Speed

* Perception: Stat.Intellect

* Healing: Stat.Intellect

* Survival: Stat.Intellect

* Stealth: Stat.Speed

* Investigation: Stat.Intellect

* Thievery: Stat.Speed



**Numenera**

* Mechanical: Stat.Intellect

* Biological: Stat.Intellect

* Crystalline: Stat.Intellect

* Energy: Stat.Intellect

* Psychic: Stat.Intellect

* Psychic: Stat.Intellect



**Crafting**

* Weapons: Stat.Intellect

* Armor: Stat.Intellect

* Devices: Stat.Intellect

* Tools: Stat.Intellect

* Consumables: Stat.Intellect

* Breeding: Stat.Intellect

* Animal Raising: Stat.Intellect



**Combat**

* Attack: Stat.Might

* Attack: Stat.Speed

* Attack: Stat.Intellect

* Defense: Stat.Might

* Defense: Stat.Speed

* Defense: Stat.Intellect

* Initiative: Stat.Speed

* Light Weapons: Stat.Speed

* Medium Weapons: Stat.Might

* Heavy Weapons: Stat.Might

In [135]:
class Character:
    def __init__(self, name='', 
                 might=0, 
                 speed=0, 
                 intellect=0, 
                 mightEdge=0, 
                 speedEdge=0, 
                 intellectEdge=0,
                 inputSkills = []):
        self.name = name
        self.might = Pool(might, mightEdge)
        self.speed = Pool(speed, speedEdge)
        self.intellect = Pool(intellect, intellectEdge)
        self.adventureSkills = []
        self.combatSkills = []
        self.craftingSkills = []
        self.numeneraSkills = []
        self.socialSkills = []
        for skill in inputSkills:
            if(skills.isSkill(skill)):
                if(skills.isAdventuringSkill(skill)):
                    self.adventureSkills.append(skill)
                elif(skills.isCombatSkill(skill)):
                    self.combatSkills.append(skill)
                elif(skills.isCraftingSkill(skill)):
                    self.craftingSkills.append(skill)
                elif(skills.isNumeneraSkill(skill)):
                    self.numeneraSkills.append(skill)
                elif(skills.isSocialSkill(skill)):
                    self.socialSkills.append(skill)
        self.skills = self.adventureSkills + \
                      self.combatSkills + \
                      self.craftingSkills + \
                      self.numeneraSkills + \
                      self.socialSkills
    def roll(self):
        return random.randint(1,20)
    def Challenge(self, level=0, skills=[], stat=Stat.Might, effort=0, assets=0):
        skillCounter = 0
        for skill in skills:
            if skill in self.skills:
                skillCounter += 1
        if(self.Effort(stat, effort)):
            target = (level - skillCounter - effort - assets)*3
        else:
            target = (level - skillCounter - assets)*3
        print('Target number: {}'.format(target))
        if(target <= 0):
            return True
        else:
            roll = self.roll()
            print('{} rolled {}.'.format(self.name, roll))
            return roll >= target
    def Effort(self, stat=Stat.Might, num=1):
        if(stat == Stat.Might):
            return self.might.effort(num)
        elif(stat == Stat.Speed):
            return self.speed.effort(num)
        elif(stat == Stat.Intellect):
            return self.intellect.effort(num)
    def __repr__(self):
        adventuring = 'ADVENTURING SKILLS:\n' + '\n'.join(self.adventureSkills) + '\n'
        combat = '\nCOMBAT SKILLS:\n' + '\n'.join(self.combatSkills) + '\n'
        crafting = '\nCRAFTING SKILLS:\n' + '\n'.join(self.craftingSkills) + '\n'
        numenera = '\nNUMENERA SKILLS:\n' + '\n'.join(self.numeneraSkills) + '\n'
        social = '\nSOCIAL SKILLS:\n' + '\n'.join(self.socialSkills) + '\n'
        skills = '\n' + adventuring + combat + crafting + numenera + social
        return "Name: {}\nMight: {}\nSpeed: {}\nIntellect: {}\nSkills: {}".format(self.name, self.might, self.speed, self.intellect, skills)

In [99]:
theron = Character('Theron', 18, 12, 8, 1, 1, 0, ['Might Defense', 'Might Attack', 'Speed Attack', 'Athletics', 'Heavy Weapons'])
theron

Name: Theron
Might: 18/18
Speed: 12/12
Intellect: 8/8
Skills: 
ADVENTURING SKILLS:
Athletics

COMBAT SKILLS:
Might Defense
Might Attack
Speed Attack
Heavy Weapons

CRAFTING SKILLS:


NUMENERA SKILLS:


SOCIAL SKILLS:


In [136]:
subotai = Character('Subotai', 14, 22, 12, 0, 3, 1, ['Stealth', 'Initiative', 'Acrobatics', 'Speed Defense', 'Light Weapons', 'Numenera', 'Bluff'])
subotai

Name: Subotai
Might: 14/14
Speed: 22/22
Intellect: 12/12
Skills: 
ADVENTURING SKILLS:
Stealth
Acrobatics

COMBAT SKILLS:
Initiative
Speed Defense
Light Weapons

CRAFTING SKILLS:


NUMENERA SKILLS:


SOCIAL SKILLS:
Bluff

In [94]:
skills.skills['Speed Attack']

Attack: Stat.Speed

In [176]:
subotai.Challenge(10, ['Speed Attack', 'Light Weapons', 'Stealth', 'Perception'], Stat.Speed, 2, 0)

Not enough to spend 2 effort.
Current: 1
Required: 3
Target number: 24
Subotai rolled 1.


False

In [118]:
subotai

Name: Subotai
Might: 14/14
Speed: 1/22
Intellect: 12/12
Skills: 
ADVENTURING SKILLS:
Stealth
Acrobatics

COMBAT SKILLS:
Initiative
Speed Defense
Light Weapons

CRAFTING SKILLS:


NUMENERA SKILLS:


SOCIAL SKILLS:
Bluff

In [31]:
Skills.isSkill('Perception')

Perception False


False

In [20]:
skillsList = [skills, numeneraSkills, craftingSkills, combatSkills]

In [21]:
skillsList

[{'Acrobatics': Acrobatics: Stat.Speed,
  'Athletics': Athletics: Stat.Might,
  'Bluff': Bluff: Stat.Intellect,
  'Diplomacy': Diplomacy: Stat.Intellect,
  'Healing': Healing: Stat.Intellect,
  'Insight': Insight: Stat.Intellect,
  'Investigation': Investigation: Stat.Intellect,
  'Perception': Perception: Stat.Intellect,
  'Perform': Perform: Stat.Intellect,
  'Stealth': Stealth: Stat.Speed,
  'Survival': Survival: Stat.Intellect},
 {'Biological': Biological: Stat.Intellect,
  'Crystalline': Crystalline: Stat.Intellect,
  'Energy': Energy: Stat.Intellect,
  'Mechanical': Mechanical: Stat.Intellect,
  'Psychic': Psychic: Stat.Intellect,
  'Temporal': Psychic: Stat.Intellect},
 {'Armor': Armor: Stat.Intellect,
  'Consumables': Consumables: Stat.Intellect,
  'Devices': Devices: Stat.Intellect,
  'Tools': Tools: Stat.Intellect,
  'Weapons': Weapons: Stat.Intellect},
 {'Initiative': Initiative: Stat.Speed,
  'Intellect Attack': Attack: Stat.Intellect,
  'Intellect Defense': Defense: Stat.I

In [9]:
Character('foo', 16, 10, 8)

Name: foo
Might: 16/16
Speed: 10/10
Intellect: 8/8

In [4]:
class Monster:
    def __init__(self, name='', level=0, hp=0, armor=0, attacks={'Might': 0, 'Speed': 0, 'Intellect': 0}, defenses={'Might': 0, 'Speed': 0, 'Intellect': 0}):
        self.name = name
        self.level = level
        self.hp = hp
        self.armor = armor
        self.attacks = attacks
    def Attack(self, target, stat):
        attackLevel = self.level+self.attacks[stat]
        target.Defend(attackLevel, stat)

In [ ]:
class Creature:
    